In [ ]:
# H1 Title 

In [ ]:
# H2 Data Collection, Cleaning, & EDA

In [ ]:
# Picture

In [ ]:
# H4 Problem statement

In [ ]:
# Problem Statement

In [ ]:
# H4 Data info

In [ ]:
# Data info and background 

In [ ]:
# H4  Subreddit 1 

In [ ]:
# Subreddit 1 info

In [ ]:
# H4 Subreddit 2 

In [ ]:
# Subreddit 2 info

In [ ]:
# Research questions

In [ ]:
# H2 Data Collection

In [ ]:
# H4 imports

In [ ]:
import requests
import pandas as pd

In [ ]:
# Scrape 

In [ ]:
# Scrape

In [ ]:
# Scrape

In [ ]:
# Scrape

In [ ]:
# Scrape

In [ ]:
# Scrape/wrangle

In [ ]:
# Make df

In [ ]:
# H2 Cleaning 

In [ ]:
# H4 imports

In [ ]:
# Imports

In [ ]:
# H4 Rename columns md

In [ ]:
# Rename cols

In [ ]:
# Check dtypes, string length, etc

In [ ]:
# H2 Exploratory Data Analysis

In [ ]:
# EDA

In [ ]:
#### H4 Chart title md

In [ ]:
# chart

In [ ]:
#### H4 Chart title md

In [ ]:
# chart

In [ ]:
params = {
    'subreddit':'news',
    'size':50,
    'q':'keyword'
}

In [ ]:
# H3 Export df

In [ ]:
# export df